In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

raw_df = pd.read_csv('../../data/merged_data_sentiment.csv')
raw_df.head()

,ID,Content,User,Date,Location,Reactions,N_Children,Post Title,Platfrom,meta,lang,Subreddit,Unnamed: 0,Sentiment
0,UgyDawiDJRU5j8e9g6x4AaABAg,i love my mum~!TAT,carol lui,2013-04-30,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCgxe0Tv...",en,NaN,NaN,"0.9998131394386292,0.9847044944763184,0.992429..."
1,Ugx5UKqZcGpX0Qi8art4AaABAg,I was in tears until the lady gave her mother ...,C Ladner,2013-05-13,NaN,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UC98Fh4X...",en,NaN,NaN,"-0.8607058525085449,-0.4791601002216339,0,0.0,..."
2,UgybIB4C2ppmGqU40WV4AaABAg,"I like her voice!! So deep, i thought the voic...",joseph jr.,2013-05-24,PH,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCUe_b0X...",en,NaN,NaN,"0.9991986155509949,0.9696046113967896,0.984618..."
3,Ugyi1lQIQnmZ2jVwk4B4AaABAg,A verve and flow equal to Maria Carrey's debut...,medusanerve,2013-05-28,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCILrk7Q...",en,NaN,NaN,"0.9990953207015991,0.901877224445343,0.9919101..."
4,Ugw2iIVWPilqFB7RMEl4AaABAg,great voice:D\n,ramon goedendorp,2013-06-08,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCNs25ls...",en,NaN,NaN,"0.9998722076416016,0.9386110305786133,0.989294..."


In [2]:
# get mean sentiment for each comment
for i, row in raw_df.iterrows():
  sent = row['Sentiment'].split(',')
  sent = [float(s) for s in sent]
  raw_df.at[i, 'Avg_Sentiment'] = sum(sent) / len(sent)

In [3]:
# show line chart of sentiment over time (by month)

# get month from date
raw_df['Month'] = pd.to_datetime(raw_df['Date']).dt.to_period('M').astype(str)

# compute volume (number of rows in the df)
raw_df['Volume'] = 1

# group by month and get mean sentiment and also sum of volume
df = raw_df.groupby('Month').agg({'Avg_Sentiment': 'mean', 'Volume': 'sum'}).reset_index()

# plot both the average sentiment and the volume (same chart, two y axes to allow for different scales)
# (secondary y-axis with subplots, use add_trace() to add the second trace)
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    go.Scatter(x=df['Month'], y=df['Avg_Sentiment'], name="Average Sentiment"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['Month'], y=df['Volume'], name="Volume"),
    secondary_y=True,
)

fig.update_layout(
    title_text="Average Sentiment and Volume by Month",
    xaxis_title="Month",
    yaxis_title="Average Sentiment",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig.show()